In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import folium
import json

# Processing transactions data

In [2]:
nodes_asn = pd.read_csv('ripple-asn-nodes.csv', delimiter=',', header=None, names=['asn','lat','lon']).dropna().drop_duplicates()

In [3]:
list_double = set()
def remove_double_asn(x):
    if(' ' in x):
        list_double.add(tuple(x.split(' ')))
        return x.split(' ')[0]
    else:
        return x
    
nodes_asn['asn'] = nodes_asn['asn'].apply(lambda x: remove_double_asn(x))

In [4]:
nodes_to_asn_dict = nodes_asn['asn'].to_dict()

In [5]:
data = pd.read_csv('txs-parsed-jan-2013-aug-2017/final_txs_2017.txt', delimiter='\n', header=None)

json_list = '['
for i in range(len(data)):
    json_text = data[0][i]
    json_list += json_text + ','
json_list = json_list[:-1]+']'

final_json = json.loads(json_list)
transactions = pd.DataFrame.from_dict(final_json)

In [6]:
transaction_nodes = set(transactions['receiver'].values)
transaction_nodes.update(transactions['sender'].values)

# Convert account public key to account id

This is a python adaptation of the js code from ripple : https://developers.ripple.com/accounts.html#address-encoding

In [8]:
import hashlib
import base58

default_base58 = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
ripple_base58 = 'rpshnaf39wBUDNEGHJKLM4PQRST7VWXYZ2bcdeCg65jkm8oFqi1tuvAxyz'

default_to_ripple = dict(zip(default_base58, ripple_base58))
ripple_to_default = dict(zip(ripple_base58, default_base58))

def key_to_account(test_key):
    pubkey_inner_hash = hashlib.sha256(test_key)
    pubkey_outer_hash = hashlib.new('ripemd160')
    pubkey_outer_hash.update(pubkey_inner_hash.digest())
    account_id = pubkey_outer_hash.digest()

    address_type_prefix = bytearray(b'\x00')
    payload = address_type_prefix + bytearray(account_id)
    chksum_hash1 = hashlib.sha256(payload).digest()
    chksum_hash2 = hashlib.sha256(chksum_hash1).digest()
    checksum = chksum_hash2[:4]
    
    dataToEncode = payload + checksum
    address = base58.b58encode(bytes(dataToEncode))
    return ''.join([default_to_ripple[letter] for letter in address.decode('utf-8')])

pubkey = bytearray.fromhex('02E23B99E7D158D0A0D5C20F8890B8365CAF4FEE9A0043B2498C3130258718F32D')
key_to_account(pubkey)

'rwSdwXsQVCjYXUyempsGxk1Sgws6perYZL'

This account id does not match the node_public key of the ripple topology. (r... VS n...)